In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))


True
12.1
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
import mne
import numpy as np

import os

import matplotlib.pyplot as plt

from sklearn.utils import resample

In [3]:
directory = './eeg-during-mental-arithmetic-tasks-1.0.0/'

rest_filepaths = []
task_filepaths = []

for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if filename.endswith('.edf'):
        label = filename.split('_')[-1].split('.')[0]

        if label == '1':
            rest_filepaths.append(filepath)
        else:
            task_filepaths.append(filepath)

In [4]:
import mne
import numpy as np
from sklearn.model_selection import train_test_split

# Example function to read and process data
def process_data(filepath):
    data = mne.io.read_raw_edf(filepath, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    
    min_t, max_t = 0, 61.99
    data.crop(tmin=min_t, tmax=max_t)
    
    tmin, tmax = 0, 0.5  # Epoch duration of 1 second
    epochs = mne.make_fixed_length_epochs(data, duration=tmax, preload=True)
    
    return epochs

# Function to process all 'task' labeled files
def process_task_files(filepaths):
    epochs_data = []  # List to store epoch data
    labels = []  # List to store corresponding labels
    
    for filepath in filepaths:
        epochs = process_data(filepath)
        epochs_data.extend(epochs.get_data())
        labels.extend([1] * len(epochs))  # Assign label 1 for 'task' (assuming 'task' label)
    
    return np.array(epochs_data), np.array(labels)

def process_rest_files(filepaths):
    epochs_data = []  # List to store epoch data
    labels = []  # List to store corresponding labels
    
    for filepath in filepaths:
        epochs = process_data(filepath)
        epochs_data.extend(epochs.get_data())
        labels.extend([0] * len(epochs))  # Assign label 1 for 'task' (assuming 'task' label)
    
    return np.array(epochs_data), np.array(labels)


In [5]:
%%capture
rest_epochs_data, rest_labels = process_rest_files(rest_filepaths)

task_epochs_data, task_labels = process_task_files(task_filepaths)

In [6]:
rest_epochs_data.shape, task_epochs_data.shape

((4428, 21, 250), (4428, 21, 250))

In [7]:
rest_labels.shape , task_labels.shape

((4428,), (4428,))

In [8]:
epochs_data_combined = np.concatenate([rest_epochs_data, task_epochs_data], axis=0)

# Concatenate labels
labels_combined = np.concatenate([rest_labels, task_labels], axis=0)

# Shuffle (optional)
# Use the same random seed for synchronizing shuffle across data and labels
random_state = 42
np.random.seed(random_state)
shuffle_indices = np.random.permutation(len(labels_combined))
data = epochs_data_combined[shuffle_indices]
label = labels_combined[shuffle_indices]

In [9]:
data.shape, label.shape

((8856, 21, 250), (8856,))

In [10]:
np.save('data.npy', data)
np.save('label.npy', label)

In [11]:
data = np.load('data.npy')
label = np.load('label.npy')

# # Convert numpy arrays to PyTorch tensors
# data = torch.tensor(data, dtype=torch.float32)
# label = torch.tensor(label, dtype=torch.long)  # Assuming labels are integers (dtype=torch.long)

# Print shapes to verify
print("Epochs tensor shape:", data.shape)
print("Labels tensor shape:", label.shape)

Epochs tensor shape: (8856, 21, 250)
Labels tensor shape: (8856,)


In [12]:
label.sum()

np.int64(4428)

In [13]:
# if isinstance(data, np.ndarray):
#     data = data[:, np.newaxis, :, :]
# else:
#     data = data.unsqueeze(1).permute(0, 1, 3, 2)

print(data.shape)

(8856, 21, 250)


In [14]:
from sklearn.model_selection import train_test_split


X_train , temp_data, y_train , temp_labels = train_test_split(data, label, test_size=0.3, random_state=42)
X_val , X_test , y_val , y_test  = train_test_split(temp_data, temp_labels, test_size=0.5, random_state=42)


### Model


In [15]:
import torch
from torch.utils.data import TensorDataset


# Choosing Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Normalizing Labels to [0, 1, 2, 3]
# y = labels - np.min(labels)
y = label

# Normalizing Input features: z-score(mean=0, std=1)
X = (data - np.mean(data)) / np.std(data)

# Checking the existance of null & inf in the dataset
if np.any(np.isnan(X)) or np.any(np.isinf(X)):
    raise ValueError("Data contains NaNs or infinities after normalization.")
if np.any(np.isnan(y)) or np.any(np.isinf(y)):
    raise ValueError("Labels contain NaNs or infinities.")

# Making the X,y tensors for K-Fold Cross Validation
X_tensor = torch.Tensor(X).unsqueeze(1)
y_tensor = torch.LongTensor(y)

# Spliting  Data: 80% for Train and 20% for Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Converting to Tensor
X_train = torch.Tensor(X_train).unsqueeze(1).to(device)
X_test = torch.Tensor(X_test).unsqueeze(1).to(device)
y_train = torch.LongTensor(y_train).to(device)
y_test = torch.LongTensor(y_test).to(device)

# Creating Tensor Dataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Printing the sizes
print("Size of X_train:", X_train.size())
print("Size of X_test:", X_test.size())
print("Size of y_train:", y_train.size())
print("Size of y_test:", y_test.size())


Size of X_train: torch.Size([7084, 1, 21, 250])
Size of X_test: torch.Size([1772, 1, 21, 250])
Size of y_train: torch.Size([7084])
Size of y_test: torch.Size([1772])


# Model


In [16]:
import torch
import torch.nn.functional as F

# Example sizes
X_train = torch.randn(7084, 1, 21, 250)  # Example training data
X_test = torch.randn(1772, 1, 21, 250)   # Example test data
y_train = torch.randint(0, 10, (7084,))  # Example training labels
y_test = torch.randint(0, 10, (1772,))   # Example test labels

# Reshape and interpolate training data
X_train_reshaped = X_train.view(-1, 21, 250)  # Reshape to (7084, 21, 250)
X_train_interpolated = F.interpolate(X_train_reshaped.unsqueeze(1), size=(224, 224), mode='bilinear', align_corners=False)  # Interpolate to (7084, 1, 224, 224)
X_train_final = torch.cat((X_train_interpolated, X_train_interpolated, X_train_interpolated), dim=1)  # Create 3 channels

# Reshape and interpolate test data
X_test_reshaped = X_test.view(-1, 21, 250)  # Reshape to (1772, 21, 250)
X_test_interpolated = F.interpolate(X_test_reshaped.unsqueeze(1), size=(224, 224), mode='bilinear', align_corners=False)  # Interpolate to (1772, 1, 224, 224)
X_test_final = torch.cat((X_test_interpolated, X_test_interpolated, X_test_interpolated), dim=1)  # Create 3 channels

# Print sizes to verify
print(f"Size of X_train_final: {X_train_final.size()}")
print(f"Size of X_test_final: {X_test_final.size()}")

Size of X_train_final: torch.Size([7084, 3, 224, 224])
Size of X_test_final: torch.Size([1772, 3, 224, 224])


In [17]:
import torch
from transformers import ViTFeatureExtractor, ViTForImageClassification
from torchvision import transforms
from PIL import Image
import requests

# # Load EEG data (replace with your actual data loading)
# # Assuming X_train_final and X_test_final are already prepared as torch tensors
# X_train_final = torch.randn(7084, 3, 224, 224)  # Example training data
# X_test_final = torch.randn(1772, 3, 224, 224)   # Example test data
# y_train = torch.randint(0, 2, (7084,))  # Example training labels (binary classification)
# y_test = torch.randint(0, 2, (1772,))   # Example test labels (binary classification)

# Initialize ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-384')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-384')

# Modify the model's classifier head for binary classification
model.classifier = torch.nn.Linear(model.config.hidden_size, 2)  # Assuming 2 classes

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example data preprocessing and loading
def preprocess_image(url):
    image = Image.open(requests.get(url, stream=True).raw)
    inputs = feature_extractor(images=image, return_tensors="pt")
    inputs = inputs.pixel_values.to(device)
    return inputs

# Example forward pass and training loop
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

# Training loop (adjust as needed)
for epoch in range(5):  # Example of 5 epochs, adjust as needed
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    inputs = X_train_final.to(device)
    labels = y_train.to(device)
    outputs = model(inputs)
    loss = criterion(outputs.logits, labels)
    
    # Backward pass and optimization step
    loss.backward()
    optimizer.step()
    
    # Print training statistics
    print(f"Epoch [{epoch+1}/5], Loss: {loss.item():.4f}")

# Evaluation (not included in training loop)
model.eval()
with torch.no_grad():
    inputs = X_test_final.to(device)
    labels = y_test.to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.logits, 1)
    accuracy = (predicted == labels).sum().item() / labels.size(0)
    print(f"Test Accuracy: {accuracy:.4f}")


c:\Users\Atharva\Anaconda3\envs\tfgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.vit.feature_extraction_vit because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:
!pip install torch torchvision transformers --upgrade

